In [1]:
import os
import shutil
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def calculate_model_metrics(observation: pd.Series, prediction: pd.Series) -> dict:
    denominator = np.sum(observation)
    if denominator == 0:
        pbias = np.nan
    else:
        pbias = 100.0 * np.sum(prediction - observation) / denominator

    metrics = {
        "r_square": r2_score(observation, prediction),
        "rmse": np.sqrt(mean_squared_error(observation, prediction)),
        "mae": mean_absolute_error(observation, prediction),
        "pbias": pbias
    }
    return metrics

In [3]:
fpath = "gtnnwr_output_20250708_222417/coefficients_20250708_222417.csv"
current_layer = "Layer_4"

dirname = os.path.dirname(fpath)


df = pd.read_csv(fpath)
df = df.rename({"Unnamed: 0":"id"}, axis=1)
stations = sorted(df["STATION"].unique())
df.head(3)

,coef_CUMDISP,bias,Pred_Layer_4,dataset_belong,denormalized_pred_result,id,STATION,X_TWD97,Y_TWD97,monthly,time,Layer_4,CUMDISP,__belong__
0,6.673480,-6.434207,-4.232186,train,-4.232186,841,NEILIAO,184142.181573,2.611723e+06,50,2020-07-01,-3.0,-201.257487,train
1,2.054093,-1.793966,-0.309616,train,-0.309616,265,FENGAN,171859.184496,2.631894e+06,2,2016-07-01,-1.0,-60.788199,train
2,4.922402,-3.792486,0.500269,train,0.500269,210,ERLUN,190429.148778,2.629865e+06,13,2017-06-01,1.0,-7.322982,train


In [4]:
summary_savefolder = os.path.join(os.getcwd(), "result_summary", dirname)
savefig_folder = os.path.join(summary_savefolder, "figs")

if not os.path.exists(savefig_folder):
    os.makedirs(savefig_folder, exist_ok=True)

stations = sorted(df["STATION"].unique())

for belong_type in ["train", "valid", "test"]:
    
    output_df = pd.DataFrame(data=None)
    
    # select_station = stations[0]
    for select_station in stations:

        df_byStation = df.query("STATION==@select_station")
        
        df_byStation = df_byStation.sort_values(by="monthly")

        temp = df_byStation.query("dataset_belong==@belong_type")

        obs_arr  = temp[current_layer]
        sim_arr = temp["denormalized_pred_result"]

        metric_df = pd.DataFrame(calculate_model_metrics(obs_arr, sim_arr), index=[select_station])
        
        metric_df["dataset_belong"] = belong_type
        
        output_df = pd.concat([output_df, metric_df])
        
    output_df.to_csv(os.path.join(summary_savefolder, f"{dirname}_{belong_type}.csv"))


In [5]:
# Step 1: Data preparation (same as before)
# select_station = "TUKU"
for select_station in stations:
    df_byStation = df.query("STATION==@select_station")
    df_byStation = df_byStation.sort_values(by="monthly").reset_index(drop=True)

    # Step 2: Get evaluation metrics for this station
    evaluation_metrics = output_df.loc[select_station]

    # Step 3: Create figure and axis using the requested method
    fig = plt.figure(figsize=(10, 3))
    ax = fig.add_subplot()

    # Step 4: Plot the data on the axis object
    ax.plot(df_byStation.index, df_byStation[current_layer], marker="o", label="Obs")
    ax.plot(
        df_byStation.index,
        df_byStation["denormalized_pred_result"],
        marker="o",
        label="Pred",
    )

    # Step 5: Set title and legend
    ax.set_title(select_station, fontweight='bold')
    ax.legend()

    # Step 6: Add evaluation metrics text in lower left corner
    metrics_text = f"""R² = {evaluation_metrics['r_square']:.3f}
    RMSE = {evaluation_metrics['rmse']:.3f}
    MAE = {evaluation_metrics['mae']:.3f}
    PBIAS = {evaluation_metrics['pbias']:.1f}%"""

    # Position text at lower left: x=0.02, y=0.02 in axis coordinates
    ax.text(
        0.02,
        0.02,
        metrics_text,
        transform=ax.transAxes,  # Use axis coordinates (0-1 range)
        verticalalignment="bottom",
        bbox=dict(boxstyle="round", facecolor="white", alpha=0.8),
    )

    savepath = os.path.join(savefig_folder, f"{select_station}png")

    plt.savefig(
        savepath,
        dpi=300,
        transparent=False,
        facecolor="w",
        edgecolor="w",
        bbox_inches="tight",
    )

    plt.close()